# **NLP com Flair**
Este tutorial é baseado no livro "Natural Language Processing with Flair".

# 4 - Otimizando Hiperparâmetros

Em uma rede neural, os "parâmetros" dos modelos podem ser vistos como os pesos em cada neurônio. Já os hiperparâmetros
são parâmetros da escolha dos modelos, por exemplo, o número de camadas que nossa rede neural vai ter.

Otimizar hiperparâmetros se trata de tentar identificar o melhor modelo variando
coisas como o número de camadas, o learning rate, funções de ativação, etc.
Para comparar os diferentes modelos usamos o validation set.

A maneira mais simples de fazer essa otimização seria simplesmente testar diferentes combinações de
parâmetros e pegar aquela com melhor resultado.
O problema dessa abordagem é que podemos ter possibilidades demais, e testar todas elas seria muito
oneroso. Assim, queremos fazer uma busca mais inteligente.
Existem diversos métodos para fazer isso. Para nos auxiliar, vamos usar o pacote `hyperopt`.

Temos 3 componentes básicos que precisamos definir quando vamos fazer uma otimização
de hiperparâmetros (3 "hiperhiperparâmetros"). Precisamos definir um
espaço de otimização (quais valores possíveis para os parâmetros que vamos variar),
a função objetivo (o que queremos maximizar ou minizar), e o método de otimização.

## Exemplo Trivial - O básico do Hyperopt

Vamos começar com um caso simples para mostrar como o `hyperopt` funciona.

Suponha que queremos simplesmente achar o mínimo de uma função.

In [9]:
def objective(value):
    return value ** 2

# O valor mínimo para essa função objetivo é claramente 0.

In [6]:
from hyperopt import hp
space = hp.uniform('param1', -100, 100)

from hyperopt import tpe
# the Tree of Parzen Estimators (TPE) algorithm
optimization_method = tpe.suggest

In [8]:
from hyperopt import fmin
# minimize the objective function over 1000 evaluations
best_params = fmin(objective,
                   space,
                   algo=optimization_method,
                   max_evals=1000)
# print the best hyperparameter value
print(best_params)

100%|███████████████| 1000/1000 [00:03<00:00, 272.73trial/s, best loss: 8.647068484854811e-05]
{'param1': 0.009298961493013513}


## Hyperopt no Flair

O Flair já possui otimização de hiperparâmetros dentro do seu pacote. Essa otimização é construída
usando o `hyperopt`.

Vamos começar importando nosso corpus e criando nossos embeddings.
Na nossa otimização, vamos testar tanto no nosso embedding com stacking
como o transformer.

In [81]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import (WordEmbeddings,
                              StackedEmbeddings,
                              FlairEmbeddings,
                              TransformerWordEmbeddings
                             )
import random
random.seed(123)  # ensure we get same split every time

# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/home/davibarreira/MEGA/resolvvi/lener-br/leNER-Br/'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train/train.conll',
                              test_file='test/test.conll',
                              dev_file='dev/dev.conll')
corpus.downsample(0.01)


import random
random.seed(123)  # ensure we get same split every time


embedding_types = [
    WordEmbeddings('pt'),
    FlairEmbeddings('pt-forward'),
    FlairEmbeddings('pt-backward'),
]
stack       = StackedEmbeddings(embeddings=embedding_types)
# transformer = TransformerWordEmbeddings('./models/BERTikal/', layers='-1', layer_mean=False)

2022-10-19 21:35:42,710 Reading data from /home/davibarreira/MEGA/resolvvi/lener-br/leNER-Br
2022-10-19 21:35:42,712 Train: /home/davibarreira/MEGA/resolvvi/lener-br/leNER-Br/train/train.conll
2022-10-19 21:35:42,713 Dev: /home/davibarreira/MEGA/resolvvi/lener-br/leNER-Br/dev/dev.conll
2022-10-19 21:35:42,714 Test: /home/davibarreira/MEGA/resolvvi/lener-br/leNER-Br/test/test.conll


---- 

O código acima já foi apresentado nos notebooks passados. Eles simplesmente
está lendo o corpus e definindo os embeddings.

Vamos agora definir a nossa otimização dos hiperparâmetros.

In [52]:
from flair.hyperparameter.param_selection import SearchSpace, Parameter
from hyperopt import hp
search_space = SearchSpace()
search_space.add(Parameter.HIDDEN_SIZE,
                 hp.choice,
                 options=[128, 256])
search_space.add(Parameter.LEARNING_RATE,
                 hp.choice,
                 options=[0.05, 0.1, 0.15, 0.2])
search_space.add(Parameter.EMBEDDINGS,
                 hp.choice,
                 options=([stack]))

A função `hp.choice` simplesmente define que o espaço de otimização é categórico. 
Note que estamos variando o número de hidden layers, o learning rate e o tipo de embedding.

O Flair requer que sempre esteja definido o learning rate e os embeddings no espaço de otimização.
Caso não queira variá-los, basta passar um único valor.

Por fim, temos que definir o objeto de seleção de parâmetros. Passamos para esse objeto o
nosso corpus, o tipo de tag (no nosso caso NER), e a pasta onde salvar os resultados.

In [53]:
from flair.hyperparameter.param_selection import SequenceTaggerParamSelector

tag_type = 'ner'
param_selector = SequenceTaggerParamSelector(corpus,
                                             tag_type,
                                             './models/paramopt') 

2022-10-19 21:25:02,511 Computing label dictionary. Progress:


78it [00:00, 26279.68it/s]

2022-10-19 21:25:02,516 Dictionary created for label 'ner' with 7 values: ORGANIZACAO (seen 36 times), JURISPRUDENCIA (seen 19 times), PESSOA (seen 15 times), TEMPO (seen 14 times), LEGISLACAO (seen 12 times), LOCAL (seen 2 times)


---- 

Estamos quase prontos para inicar o treino. Para isso,
basta usar o método `optimize` do objeto `param_selector`. Porém, precisamos passar
o `max_evals`. Esse valor define o máximo de combinações que vamos testar.
No nosso caso, temos uma quantidade finita de possibilidades. Mas caso
tivéssemos usado, por exemplo, `hp.uniform` no learning rate,
teríamos infinitas possibilidades.

Só para exemplificar, vamos usar `max_evals = 4`. Logo, vamos treinar 4 modelos diferentes.
A escolha fica por conta do Flair, que já escolheu o método de otimização e a função objetivo (que é a loss do modelo).

In [50]:
param_selector.optimize(search_space, max_evals = 4)

  0%|                                                   | 0/4 [00:00<?, ?trial/s, best loss=?]2022-10-19 21:23:42,705 ----------------------------------------------------------------------------------------------------
2022-10-19 21:23:42,710 Evaluation run: 1
2022-10-19 21:23:42,711 Evaluating parameter combination:
2022-10-19 21:23:42,713 	embeddings: TransformerWordEmbeddings(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768

job exception: The expanded size of the tensor (542) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [32, 542].  Tensor sizes: [1, 512]



  0%|                                                   | 0/4 [00:00<?, ?trial/s, best loss=?]


RuntimeError: The expanded size of the tensor (542) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [32, 542].  Tensor sizes: [1, 512]

In [18]:
with open('./models/paramopt/param_selection.txt') as f:
    lines = f.readlines()

In [22]:
print('\n'.join(lines))

evaluation run 1

	embeddings: StackedEmbeddings [0-/home/davibarreira/.flair/embeddings/pt-wiki-fasttext-300d-1M,1-/home/davibarreira/.flair/embeddings/lm-pt-forward.pt,2-/home/davibarreira/.flair/embeddings/lm-pt-backward.pt]

	hidden_size: 128

	learning_rate: 0.05

loss: 0.2694319188594818

variance: 5.557327042993165e-09

test_score: 0.12903225806451613

----------------------------------------------------------------------------------------------------

